In [3]:
from fastmcp import Client

url = "http://34.222.253.74:8080"
# url = "http://127.0.0.1:8000"

# async def main():
async with Client(f"{url}/mcp") as client:
    tools = await client.list_tools()
    print(f"Available tools: {tools}")
    result = await client.call_tool("add", {"a": 5190283, "b": 9})
    print(f"Result: {result[0].text}")
    
    result = await client.call_tool("query_fundamental_analysis", {"sql_query": "SELECT Papel, DivYield, P_L FROM fundamental_analysis WHERE DivYield > 9"})
    print(f"Result: {result[0].text}")
        


Available tools: [Tool(name='query_fundamental_analysis', description='Query a SQL Database about Fundamental Analysis of the Brazilian Stock market with the following schema:\n    \nCREATE TABLE fundamental_analysis (\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n\tPapel VARCHAR(10) NOT NULL, # Código da Ação\n\tCotacao FLOAT NOT NULL, # Preço da Ação\n\tTipo VARCHAR(10) NOT NULL, # Tipo da Ação\n\tDataUltimaCotacao DATETIME NOT NULL, # Data da última cotação\n\tEmpresa VARCHAR(100) NOT NULL, # Nome da Empresa\n\tMin52Sem FLOAT NOT NULL, # Cotação Mínima nas últimas 52 semanas\n\tSetor VARCHAR(100) NOT NULL, # Setor da Empresa\n\tMax52Sem FLOAT NOT NULL, # Cotação Máxima nas últimas 52 semanas\n\tSubsetor VARCHAR(100) NOT NULL, # Subsetor da Empresa\n\tVolumeMedio2M FLOAT NOT NULL, # Volume médio de negociações nos últimos 2 meses\n\tValorMercado FLOAT NOT NULL, # Valor de Mercado da Empresa\n\tUltimoBalancoProcessado DATETIME NOT NULL, # Último Balanço patrimonial Processado\n\tValorFi

In [1]:
import sqlite3

database_path = "fundamental_analysis/2025-07-18/fundamental_analysis.db"

with open(database_path, "w") as f:
    f.write("")

command = """CREATE TABLE IF NOT EXISTS fundamental_analysis (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
	Papel VARCHAR(10) NOT NULL, 
	Cotacao FLOAT NOT NULL, 
	Tipo VARCHAR(10) NOT NULL, 
	DataUltimaCotacao DATETIME NOT NULL,
	Empresa VARCHAR(100) NOT NULL,
	Min52Sem FLOAT NOT NULL,
	Setor VARCHAR(100) NOT NULL,
	Max52Sem FLOAT NOT NULL,
	Subsetor VARCHAR(100) NOT NULL,
	VolumeMedio2M FLOAT NOT NULL,
	ValorMercado FLOAT NOT NULL,
	UltimoBalancoProcessado DATETIME NOT NULL,
	ValorFirma FLOAT NOT NULL,
	NumeroAcoes INTEGER NOT NULL,
	CrescimentoReceita5a FLOAT NOT NULL,
	Ativo FLOAT NOT NULL,
	DividaBruta FLOAT NOT NULL,
	Disponibilidades FLOAT NOT NULL,
	DividaLiquida FLOAT NOT NULL,
	AtivoCirculante FLOAT NOT NULL,
	PatrimonioLiquido FLOAT NOT NULL,
	ReceitaLiquida FLOAT NOT NULL,
	ReceitaLiquida3Meses FLOAT NOT NULL,
	EBIT FLOAT NOT NULL,
	EBIT3Meses FLOAT NOT NULL,
	LucroLiquido FLOAT NOT NULL,
	LucroLiquido3Meses FLOAT NOT NULL,
	DivYield FLOAT NOT NULL,
	Depositos FLOAT NOT NULL,
	CartaoCredito FLOAT NOT NULL,
	ResultadoIntFinanceiro FLOAT NOT NULL,
	ResultadoIntFinanceiro3Meses FLOAT NOT NULL,
	ReceitaServicos FLOAT NOT NULL,
	ReceitaServicos3Meses FLOAT NOT NULL
)"""

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
cursor.executescript(command)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

In [2]:
import pandas as pd
import json

df = pd.read_csv("fundamental_analysis/2025-07-18/fundamental_analysis.csv")
df = df.fillna(0)

with open("fundamental_analysis_columns.json") as f:
    column_maps = json.load(f)

command = []
for (idx, row) in df.iterrows():
    keys, values = [], []
    for column in column_maps:
        keys.append(column_maps[column])
        values.append(f"'{row[column]}'")
        
    query = f"INSERT INTO fundamental_analysis ({', '.join(keys)}) VALUES ({', '.join(values)});"
    command.append(query)
    
command = "\n".join(command)

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
cursor.executescript(command)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


In [3]:
command = "SELECT Papel, DivYield FROM fundamental_analysis WHERE DivYield > 9"

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Query the database
cursor.execute(command)

# Fetch column names
columns = [description[0] for description in cursor.description] if cursor.description else []

# Fetch results
rows = cursor.fetchall()

# Close the cursor and connection
cursor.close()
connection.close()

# Format results as markdown table
if not rows:
    output = "No data found with given query"
else:
    # Create markdown table
    output = "| " + " | ".join(columns) + " |\n"
    output += "| " + " | ".join(["---" for _ in columns]) + " |\n"
    for row in rows:
        output += "| " + " | ".join([str(cell) if cell is not None else "" for cell in row]) + " |\n"
        
output

'| Papel | DivYield |\n| --- | --- |\n| CSMG3 | 12.0 |\n| MRFG3 | 14.5 |\n| ALLD3 | 19.0 |\n| CGRA3 | 9.9 |\n| VVEO3 | 10.9 |\n| LAVV3 | 10.4 |\n| MTRE3 | 12.9 |\n| DIRR3 | 9.7 |\n| EVEN3 | 15.5 |\n| ANIM3 | 15.2 |\n| EKTR4 | 12.5 |\n| SCAR3 | 20.9 |\n| SYNE3 | 123.4 |\n| ALOS3 | 9.3 |\n| BBAS3 | 9.5 |\n| KEPL3 | 10.9 |\n| CMIN3 | 13.8 |\n| VBBR3 | 13.2 |\n| RECV3 | 19.6 |\n| TIMS3 | 9.2 |\n| STBP3 | 20.2 |\n| UCAS3 | 11.2 |\n| JBSS3 | 39.3 |\n| MRFG3 | 13.2 |\n| LEVE3 | 9.2 |\n| CGRA4 | 11.7 |\n| CGRA3 | 11.2 |\n| ALLD3 | 18.6 |\n| EPAR3 | 37.3 |\n| MTRE3 | 13.8 |\n| DIRR3 | 9.3 |\n| MELK3 | 35.1 |\n| SOND6 | 16.5 |\n| SOND5 | 15.3 |\n| ANIM3 | 14.3 |\n| ISAE4 | 10.5 |\n| CMIG4 | 12.6 |\n| EQPA3 | 11.7 |\n| EQMA3B | 9.3 |\n| EQPA5 | 10.2 |\n| EKTR3 | 12.1 |\n| CEBR3 | 10.2 |\n| CEBR5 | 10.1 |\n| EQPA6 | 9.3 |\n| CEBR6 | 10.3 |\n| EKTR4 | 10.2 |\n| GEPA3 | 21.0 |\n| GEPA4 | 19.5 |\n| SCAR3 | 10.4 |\n| SYNE3 | 108.2 |\n| LOGG3 | 9.9 |\n| HBTS5 | 20.3 |\n| MOAR3 | 38.0 |\n| BAZA3 | 11.2 

In [4]:
try:
    with sqlite3.connect(database_path) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT sql
            FROM sqlite_master
            WHERE type='table' AND sql IS NOT NULL
        """
        )
        schemas = cursor.fetchall()
        schema = "\n".join(schema[0] for schema in schemas if schema[0])
except Exception as e:
    raise Exception(f"Failed to get table schema: {e}")

print(schema)

CREATE TABLE fundamental_analysis (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
	Papel VARCHAR(10) NOT NULL, 
	Cotacao FLOAT NOT NULL, 
	Tipo VARCHAR(10) NOT NULL, 
	DataUltimaCotacao DATETIME NOT NULL,
	Empresa VARCHAR(100) NOT NULL,
	Min52Sem FLOAT NOT NULL,
	Setor VARCHAR(100) NOT NULL,
	Max52Sem FLOAT NOT NULL,
	Subsetor VARCHAR(100) NOT NULL,
	VolumeMedio2M FLOAT NOT NULL,
	ValorMercado FLOAT NOT NULL,
	UltimoBalancoProcessado DATETIME NOT NULL,
	ValorFirma FLOAT NOT NULL,
	NumeroAcoes INTEGER NOT NULL,
	CrescimentoReceita5a FLOAT NOT NULL,
	Ativo FLOAT NOT NULL,
	DividaBruta FLOAT NOT NULL,
	Disponibilidades FLOAT NOT NULL,
	DividaLiquida FLOAT NOT NULL,
	AtivoCirculante FLOAT NOT NULL,
	PatrimonioLiquido FLOAT NOT NULL,
	ReceitaLiquida FLOAT NOT NULL,
	ReceitaLiquida3Meses FLOAT NOT NULL,
	EBIT FLOAT NOT NULL,
	EBIT3Meses FLOAT NOT NULL,
	LucroLiquido FLOAT NOT NULL,
	LucroLiquido3Meses FLOAT NOT NULL,
	DivYield FLOAT NOT NULL,
	Depositos FLOAT NOT NULL,
	CartaoCredito FLOAT NOT N